In [ ]:
"""
文泉书局
"""

In [ ]:
"""
Get images
"""
import requests
import time
import jwt
import json

BOOK_ID = '5611'
PAGE_NUM = 448
NAME = 'math' # used as images directory name & pdf name

# get key
k_url = f'https://lib-tsinghua.wqxuetang.com/v1/read/k?bid={BOOK_ID}'
k = requests.get(k_url).content.decode()
k = json.dumps(json.loads(k)['data'])

# jwt
SECRET = 'g0NnWdSE8qEjdMD8a1aq12qEYphwErKctvfd3IktWHWiOBpVsgkecur38aBRPn2w'
iat = int(time.time())
t = iat * 1000
encoded_jwt = jwt.encode({
    'p': '1',
    't': t,
    'b': BOOK_ID,
    'w': 1000,
    'k': k,
    'iat': iat,
}, SECRET, algorithm='HS256').decode()

url = f'https://lib-tsinghua.wqxuetang.com/page/lmg/{BOOK_ID}/{{}}?k={encoded_jwt}'

for i in range(1, PAGE_NUM+1):
    url_i = url.format(i)
    r = requests.get(url_i).content
    if not r:
        raise Exception('no content')
    with open(f'{NAME}/{i}.jpg', 'wb') as f:
        f.write(r)
    print(i, end=' ')
    time.sleep(2)

In [ ]:
"""
Generate PDF
"""

import img2pdf

images = [f'{NAME}/{i}.jpg' for i in range(1, PAGE_NUM+1)]

with open(f'{NAME}.pdf', 'wb') as f:
    f.write(img2pdf.convert(images))

# add bookmarks

import requests
tree_url = f'https://lib-tsinghua.wqxuetang.com/v1/book/catatree?bid={BOOK_ID}'
tree = requests.get(tree_url).json()['data']

from PyPDF2 import PdfFileWriter, PdfFileReader
output = PdfFileWriter() # open output
input = PdfFileReader(open(f'{NAME}.pdf', 'rb')) # open input

for i in range(PAGE_NUM):
    output.addPage(input.getPage(i)) # insert page

for chapter in tree:
    chapter_mark = output.addBookmark(chapter['label'], int(chapter['pnum'])-1, parent=None)
    if 'children' in chapter:
        for section in chapter['children']:
            output.addBookmark(section['label'], int(section['pnum'])-1, parent=chapter_mark)

with open(f'{NAME}_bookmarks.pdf', 'wb') as f:
    output.write(f)